In [1]:
import pandas as pd # standard naming convention
import numpy as np

This exercise consists in analyzing a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a pair of FPGAs. Each measurement (i.e. each row of the input file) consists of a flag that specifies the type of message ('HEAD', which in this case is always 1), two addresses of the TDC providing the signal ('FPGA' and 'TDC_CHANNEL'), and the timing information ('ORBIT_CNT', 'BX_COUNTER', and 'TDC_MEAS'). Each TDC count corresponds to 25/30 ns, whereas a unit of BX_COUNTER corresponds to 25 ns, and the ORBIT_CNT is increased every 'x' BX_COUNTER. This allows to store the time in a similar way to hours, minutes and seconds.

In [2]:
#If haven't downloaded it yet, please get the data file with wget
!wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P ./data/

/bin/bash: wget: command not found


1\. Create a Pandas DataFrame reading N rows of the `data/data_000637.txt` dataset. Choose N to be smaller than or equal to the maximum number of rows and larger that 10k (check the documentation).

In [13]:
file_name = "./data/data_000637.txt"
data = pd.read_csv(file_name)

N=100

data_N=data.head(N)

data_N

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS
0,1,0,123,3869200167,2374,26
1,1,0,124,3869200167,2374,27
2,1,0,63,3869200167,2553,28
3,1,0,64,3869200167,2558,19
4,1,0,64,3869200167,2760,25
...,...,...,...,...,...,...
95,1,0,64,3869200168,1506,4
96,1,1,6,3869200168,1503,6
97,1,0,61,3869200168,1609,10
98,1,0,59,3869200168,1614,16


2\. Estimate the number of BX in a ORBIT (the value 'x').

In [14]:
x=[]
ind=0
for i in range(0,N-1):
    if(data_N['ORBIT_CNT'][i]==data_N['ORBIT_CNT'][i+1]):
        ind+=1
    else:
        x.append(data_N['BX_COUNTER'][i]-data_N['BX_COUNTER'][i-ind])
        ind=0
        
        
x_est=np.mean(x)
print("Estimation: ",x_est)


Estimation:  813.0


3\. Find out the duration of the data taking in hours, minutes and seconds. You can either make an estimate based on the fraction of the measurements (rows) you read, or perform this check precisely by reading the whole dataset.

In [28]:

time=[]
time.append((25/30)*data_N['TDC_MEAS'][0])
for i in range(0,N-1):
    time.append((data_N['ORBIT_CNT'][i+1]-data_N['ORBIT_CNT'][i])*x_est+(data_N['BX_COUNTER'][i+1]-data_N['BX_COUNTER'][i])*25+(data_N['TDC_MEAS'][i+1]-data_N['TDC_MEAS'][i])*(25/30))
time=np.array(time)
single_s=time*pow(10,-9)
tot_s=np.sum(tot_s)
tot_m=tot_s/60
tot_h=tot_m/60
print("Estimation second",tot_s)
print("Estimation minute",tot_m)
print("Estimation hours",tot_h)

Estimation second -1.8137000000000013e-05
Estimation minute -3.0228333333333356e-07
Estimation hours -5.03805555555556e-09


4\. Create a new column with the absolute time in ns (as a combination of the other three columns with timing information) since the beginning of the data acquisition.

In [33]:
data_N['TIME']= [np.sum(time[i]) for i in range(0,N)]
data_N

<ipython-input-33-2e9a168b8705>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_N['TIME']= [np.sum(time[i]) for i in range(0,N)]


,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,TIME
0,1,0,123,3869200167,2374,26,21.666667
1,1,0,124,3869200167,2374,27,0.833333
2,1,0,63,3869200167,2553,28,4475.833333
3,1,0,64,3869200167,2558,19,117.500000
4,1,0,64,3869200167,2760,25,5055.000000
...,...,...,...,...,...,...,...
95,1,0,64,3869200168,1506,4,-42.500000
96,1,1,6,3869200168,1503,6,-73.333333
97,1,0,61,3869200168,1609,10,2653.333333
98,1,0,59,3869200168,1614,16,130.000000


5\. Use the `.groupby()` method to find out the noisy channels, i.e. the TDC channels with most counts (print to screen the top 3 and the corresponding counts)

In [50]:
max_TDC=data_N.groupby(['TDC_CHANNEL']).max()
print(max_TDC[len(max_TDC):len(max_TDC)-4:-1])

             HEAD  FPGA   ORBIT_CNT  BX_COUNTER  TDC_MEAS        TIME
TDC_CHANNEL                                                          
139             1     1  3869200168        3186         0  260.833333
128             1     1  3869200168         270         1  -34.166667
127             1     1  3869200168         273         6  238.333333


6\. Count the number of non-empty orbits (i.e. the number of orbits with at least one hit). Count also the number of unique orbits with at least one measurement from TDC_CHANNEL=139.

7\. **Optional:** Make two occupancy plots (one for each FPGA), i.e. plot the number of counts per TDC channel